In [1]:
#import libraries
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
file_path= 'data/cleaned_imdb.csv'
movies = pd.read_csv(file_path, index_col=0)
movies.head()

,movie,genre,labelled_genre,summary
0,Black Panther: Wakanda Forever,"Action,Adventure,Drama",Action,peopl wakanda fight protect home interven worl...
1,Enola Holmes 2,"Action,Adventure,Crime",Action,detect hire enola holm take first offici case ...
2,Weird: The Al Yankovic Story,"Biography,Comedy,Music",Comedy,explor everi facet yankov life meteor rise fam...
3,The Whale,Drama,Drama,reclus english teacher attempt reconnect estra...
4,Don't Worry Darling,"Drama,Thriller",Drama,housewif live husband utopian experiment commu...


In [5]:
movies.describe()

,movie,genre,labelled_genre,summary
count,64014,64014,64014,64014
unique,64014,1324,5,63789
top,Black Panther: Wakanda Forever,Drama,Drama,plot unknown
freq,1,4627,20035,80


In [ ]:
movies=movies.drop_duplicates(subset='movie')
movies = movies[movies["summary"].str.contains("Add a Plot") == False]
movies = movies.reset_index(drop=True)
movies

In [ ]:
def clean_dataset(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [ ]:
movies['clean_summary'] = movies['summary'].apply(lambda x: clean_dataset(x))
movies.head(5)

In [ ]:
del movies['summary']

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

movies['clean_summary'] = movies['clean_summary'].apply(lambda x: remove_stopwords(x))
movies.head(5)

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

movies['stemmed_summary'] = movies['clean_summary'].str.split()
movies['stemmed_summary'] = movies['stemmed_summary'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
movies['stemmed_summary'] = movies['stemmed_summary'].apply(lambda x: ' '.join(x))
movies

In [ ]:
del movies['clean_summary']
movies['summary'] = movies['stemmed_summary']
del movies['stemmed_summary']
movies

In [ ]:
movies['labelled_genre'] = movies['genre'].str.split(',')
movies['labelled_genre'] = movies['labelled_genre'].apply(lambda x: x[0])
movies

In [ ]:
movies.labelled_genre.value_counts().sort_values().plot(kind = 'barh')

In [ ]:
interested = ['Drama', 'Comedy', 'Action', 'Documentary', 'Horror']
genre_map = {
    'Adventure': 'Action', 
    'Biography': 'Documentary',
    'Crime': 'Action',
    'Animation': 'Drama',
    'Thriller': 'Horror',
    'Western': 'Drama',
    'Romance': 'Drama',
    'Fantasy': 'Horror',
    'Sci-Fi': 'Action',
    'Mystery': 'Horror',
    'Family': 'Drama',
    'Musical': 'Comedy',
    'Music': 'Comedy',
    'History': 'Documentary',
    'War': 'Action',
    'Film-Noir': 'Action',
    'Sport': 'Action',
    'Short': 'Drama',
    'Reality-TV': 'Documentary',
    'Adult': 'Comedy'
    }
new_genres = []

for row in movies['genre'].str.split(','):
    flag=0
    for genre in row:
        if genre in interested:
            new_genres.append(genre)
            flag=1
            break
    if flag == 0:
        new_genres.append(genre_map[row[0]])
        
movies['new_genre'] = new_genres
movies

In [ ]:
movies.new_genre.value_counts().sort_values().plot(kind = 'barh')

In [ ]:
movies['labelled_genre'] = movies['new_genre']
del movies['new_genre']
movies.to_csv('data/new_cleaned_imdb.csv')